In [3]:
import cv2
import os
import pandas as pd
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import numpy as np

In [2]:
# Crear dataframes de train y test através de las carpetas de fotos

train_data = []

for label in os.listdir('train'):
    if label.startswith('.'):
        continue
    folder_path = os.path.join('train',label)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        with Image.open(image_path) as image:
            pixels = np.array(image).astype(float)
            train_data.append((pixels, label))
            
            
test_data = []

for label in os.listdir('test'):
    if label.startswith('.'):
        continue
    folder_path = os.path.join('test',label)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        with Image.open(image_path) as image:
            pixels = np.array(image).astype(float)
            test_data.append((pixels, label))
            
            
df_train = pd.DataFrame(train_data, columns=['pixels', 'label'])
df_test = pd.DataFrame(test_data, columns=['pixels', 'label'])  


In [4]:

# Asignar 'X' y 'y'

X_train = df_train['pixels']
y_train = df_train['label']


# Normalizar los datos

X_train = X_train/255.0


# Convertir etiquetas en numeros

le = preprocessing.LabelEncoder()
#y_train = pd.get_dummies(y_train)
lab = len(df_train['label'])
y_train = le.fit_transform(y_train)
y_train = tf.one_hot(y_train, lab)
y_train = np.array(y_train)

2022-12-21 21:14:03.459521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:

width, height = 48, 48

# Definir Modelo
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(width, height)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(lab, activation='softmax')
])

# Compilar los datos
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, batch_size=32, epochs=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).